# [Advent of Code 2021: Day 3](https://adventofcode.com/2021/day/3)
[puzzle input](https://adventofcode.com/2021/day/3/input)

## \-\-\- Day 3: Binary Diagnostic \-\-\-

The submarine has been making some odd creaking noises, so you ask it to produce a diagnostic report just in case.

The diagnostic report (your puzzle input) consists of a list of binary numbers which, when decoded properly, can tell you many useful things about the conditions of the submarine. The first parameter to check is the **power consumption**.

You need to use the binary numbers in the diagnostic report to generate two new binary numbers (called the **gamma rate** and the **epsilon rate**). The power consumption can then be found by multiplying the gamma rate by the epsilon rate.

Each bit in the gamma rate can be determined by finding the **most common bit in the corresponding position** of all numbers in the diagnostic report. For example, given the following diagnostic report:

```
00100
11110
10110
10111
10101
01111
00111
11100
10000
11001
00010
01010
```

Considering only the first bit of each number, there are five `0` bits and seven `1` bits. Since the most common bit is `1`, the first bit of the gamma rate is `1`.

The most common second bit of the numbers in the diagnostic report is `0`, so the second bit of the gamma rate is `0`.

The most common value of the third, fourth, and fifth bits are `1`, `1`, and `0`, respectively, and so the final three bits of the gamma rate are `110`.

So, the gamma rate is the binary number `10110`, or **`22`** in decimal.

The epsilon rate is calculated in a similar way; rather than use the most common bit, the least common bit from each position is used. So, the epsilon rate is `01001`, or **`9`** in decimal. Multiplying the gamma rate (`22`) by the epsilon rate (`9`) produces the power consumption, **`198`**.

Use the binary numbers in your diagnostic report to calculate the gamma rate and epsilon rate, then multiply them together. **What is the power consumption of the submarine?** (Be sure to represent your answer in decimal, not binary.)

In [1]:
import unittest
import numpy as np
from IPython.display import Markdown, display

from aoc_puzzle import AocPuzzle

class Puzzle(AocPuzzle):
    
    def parse_data(self, raw_data):
        self.data = list(raw_data.split('\n'))
        
    def get_max_min(self, data, pos, val_when_equal='1'):
        data = list(map(list, data))
        pos_data = list(map(''.join, np.transpose(data)))
        num_1_count = pos_data[pos].count('1')
        num_0_count = pos_data[pos].count('0')
        if num_1_count > num_0_count:
            amax = '1'
            amin = '0'
        elif num_1_count == num_0_count:
            amax = val_when_equal
            amin = val_when_equal
        else:
            amax = '0'
            amin = '1'
        return (amax, amin)
            
    def run(self, output=False):
        
        gamma_rate = '0b'
        epsilon_rate = '0b'
        for i in range(len(self.data[0])):
            amax, amin = self.get_max_min(self.data, i)
            gamma_rate += amax
            epsilon_rate += amin                
            
        gamma_rate = int(gamma_rate,2)
        epsilon_rate = int(epsilon_rate,2)
        
        result = gamma_rate * epsilon_rate
        if output:
            display(Markdown(f'### Result is `{result}`'))            
        return result
        

class TestBasic(unittest.TestCase):

    def test_parse_data(self):
        in_data = '00100\n11110\n10110\n10111\n10101\n01111\n00111\n11100\n10000\n11001\n00010\n01010'
        exp_out = ['00100', '11110', '10110', '10111', '10101', '01111', '00111', '11100', '10000', '11001', '00010', '01010']
        puzzle = Puzzle(in_data)
        self.assertEqual(puzzle.data, exp_out)
        
    def test_puzzle(self):
        input_data = ['00100\n11110\n10110\n10111\n10101\n01111\n00111\n11100\n10000\n11001\n00010\n01010']
        exp_output = [198]
        for in_data, exp_out in tuple(zip(input_data, exp_output)):
            puzzle = Puzzle(in_data)
            self.assertEqual(puzzle.run(), exp_out)
        
unittest.main(argv=[""], exit=False)

..
----------------------------------------------------------------------
Ran 2 tests in 0.003s

OK


In [2]:
puzzle = Puzzle("input/d03.txt")
puzzle.run(output=True)

### Result is `693486`

693486

## \-\-\- Part Two \-\-\-

Next, you should verify the **life support rating**, which can be determined by multiplying the **oxygen generator rating** by the **CO2 scrubber rating**.

Both the oxygen generator rating and the CO2 scrubber rating are values that can be found in your diagnostic report \- finding them is the tricky part. Both values are located using a similar process that involves filtering out values until only one remains. Before searching for either rating value, start with the full list of binary numbers from your diagnostic report and **consider just the first bit** of those numbers. Then:

*   Keep only numbers selected by the **bit criteria** for the type of rating value for which you are searching. Discard numbers which do not match the bit criteria.
*   If you only have one number left, stop; this is the rating value for which you are searching.
*   Otherwise, repeat the process, considering the next bit to the right.

The **bit criteria** depends on which type of rating value you want to find:

*   To find **oxygen generator rating**, determine the **most common** value (`0` or `1`) in the current bit position, and keep only numbers with that bit in that position. If `0` and `1` are equally common, keep values with a **`1`** in the position being considered.
*   To find **CO2 scrubber rating**, determine the **least common** value (`0` or `1`) in the current bit position, and keep only numbers with that bit in that position. If `0` and `1` are equally common, keep values with a **`0`** in the position being considered.

For example, to determine the **oxygen generator rating** value using the same example diagnostic report from above:

*   Start with all 12 numbers and consider only the first bit of each number. There are more `1` bits (7) than `0` bits (5), so keep only the 7 numbers with a `1` in the first position: `11110`, `10110`, `10111`, `10101`, `11100`, `10000`, and `11001`.
*   Then, consider the second bit of the 7 remaining numbers: there are more `0` bits (4) than `1` bits (3), so keep only the 4 numbers with a `0` in the second position: `10110`, `10111`, `10101`, and `10000`.
*   In the third position, three of the four numbers have a `1`, so keep those three: `10110`, `10111`, and `10101`.
*   In the fourth position, two of the three numbers have a `1`, so keep those two: `10110` and `10111`.
*   In the fifth position, there are an equal number of `0` bits and `1` bits (one each). So, to find the **oxygen generator rating**, keep the number with a `1` in that position: `10111`.
*   As there is only one number left, stop; the **oxygen generator rating** is `10111`, or **`23`** in decimal.

Then, to determine the **CO2 scrubber rating** value from the same example above:

*   Start again with all 12 numbers and consider only the first bit of each number. There are fewer `0` bits (5) than `1` bits (7), so keep only the 5 numbers with a `0` in the first position: `00100`, `01111`, `00111`, `00010`, and `01010`.
*   Then, consider the second bit of the 5 remaining numbers: there are fewer `1` bits (2) than `0` bits (3), so keep only the 2 numbers with a `1` in the second position: `01111` and `01010`.
*   In the third position, there are an equal number of `0` bits and `1` bits (one each). So, to find the **CO2 scrubber rating**, keep the number with a `0` in that position: `01010`.
*   As there is only one number left, stop; the **CO2 scrubber rating** is `01010`, or **`10`** in decimal.

Finally, to find the life support rating, multiply the oxygen generator rating (`23`) by the CO2 scrubber rating (`10`) to get **`230`**.

Use the binary numbers in your diagnostic report to calculate the oxygen generator rating and CO2 scrubber rating, then multiply them together. **What is the life support rating of the submarine?** (Be sure to represent your answer in decimal, not binary.)

In [3]:
class Puzzle2(Puzzle):     
    
    def get_data_subset(self, data, pos, val):
        new_data = []
        for item in data:
            if item[pos] == val:
                new_data.append(item)
        return new_data        
    
    def get_val(self, val_type):
        data = self.data.copy()
        for i in range(len(data[0])):
            if len(data) == 1:
                break
            if val_type == 'O2':
                amax, amin = self.get_max_min(data, i, '1')
                data = self.get_data_subset(data, i, amax)
            elif val_type == 'CO2':                
                amax, amin = self.get_max_min(data, i, '0')
                data = self.get_data_subset(data, i, amin)
            else:
                return -1
                
        bin_str = '0b'+''.join(data[0])
        return int(bin_str,2)
    
    def run(self, output=False):
        
        o2_val = self.get_val('O2')
        co2_val = self.get_val('CO2')
                     
        result = o2_val*co2_val
        if output:
            display(Markdown(f'### Result is `{result}`'))            
        return result


class TestBasic(unittest.TestCase):
        
    def test_puzzle2(self):
        input_data = ['00100\n11110\n10110\n10111\n10101\n01111\n00111\n11100\n10000\n11001\n00010\n01010']
        exp_output = [230]
        for in_data, exp_out in tuple(zip(input_data, exp_output)):
            puzzle = Puzzle2(in_data)
            self.assertEqual(puzzle.run(), exp_out)
        
unittest.main(argv=[""], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


In [4]:
puzzle = Puzzle2("input/d03.txt")
puzzle.run(output=True)

### Result is `3379326`

3379326